#### Identify Present Letters:
Make guesses using each letter of the alphabet.
For each guess, check the API response to see if the letter is "present" or "correct."
Collect all letters that are identified as present into a set.
#### Guessing the Word:
Initialize the current state of the word with placeholders (e.g., underscores).
Create a dictionary to track confirmed letter positions.
#### Make Guesses:
Loop through each present letter:
Guess the word by repeating the letter for the entire word length.
Check the API response and update the position dictionary with any letters that are confirmed as correct.

In [46]:
import requests
import string

def guess_word(api_url, guess, seed, size=5):
    """
    Function to guess a word against the Wordle-like API.
    
    Parameters:
    - api_url: str : The API endpoint for guessing words
    - guess: str : The word to guess
    - seed: int : The seed for the random word
    - size: int : The size of the word (default is 5)
    
    Returns:
    - response: dict : The JSON response from the API
    """
    params = {
        'guess': guess,
        'seed': seed,
        'size': size
    }
    
    response = requests.get(api_url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def find_present_letters(api_url, seed, size):
    present_letters = set()
    alphabet = string.ascii_lowercase  # Initialize letters from a to z

    for letter in alphabet:
        guess = letter * size  # Example: 'aaaaaa', 'bbbbbb', etc.
        result = guess_word(api_url, guess, seed, size)

        if result and any(res['result'] in ['present', 'correct'] for res in result):
            present_letters.add(letter)

    return present_letters

def guess_with_letter(api_url, letter, seed, size, current_state, position_dict):
    """
    Function to guess a word using a specific letter and update the position dictionary.

    Parameters:
    - api_url: str : The API endpoint for guessing words
    - letter: str : The letter to guess
    - seed: int : The seed for the random word
    - size: int : The size of the word
    - current_state: list : Current state of the guessed word
    - position_dict: dict : Dictionary of confirmed letter positions

    Returns:
    - bool: Indicates if the correct word was found
    """
    guess = letter * size
    result = guess_word(api_url, guess, seed, size)
    print(f"Guess: {guess}, Response: {result}")

    if result:
        for index, res in enumerate(result):
            if res['result'] == 'correct':
                position_dict[index] = letter  # Update confirmed positions

        # Update current state with confirmed letters
        for idx, char in position_dict.items():
            current_state[idx] = char

        return len(position_dict) == size  # Check if all positions are filled

    return False

def fill_unconfirmed_positions(api_url, present_letters, size, current_state, position_dict, seed):
    """
    Function to fill in unconfirmed positions with present letters.

    Parameters:
    - api_url: str : The API endpoint for guessing words
    - present_letters: set : Set of letters that are present in the word
    - size: int : The size of the word
    - current_state: list : Current state of the guessed word
    - position_dict: dict : Dictionary of confirmed letter positions
    - seed: int : The seed for the random word
    """
    for idx in range(size):
        if idx not in position_dict:
            for letter in present_letters:
                new_guess = current_state[:]
                new_guess[idx] = letter  # Place the letter in the current position
                guess_str = ''.join(new_guess)

                result = guess_word(api_url, guess_str, seed, size)
                print(f"New Guess: {guess_str}, Response: {result}")

                if result:
                    for index, res in enumerate(result):
                        if res['result'] == 'correct':
                            position_dict[index] = letter
                            current_state[index] = letter

def main(api_url, seed, size=5):
    # Step 1: Find present letters
    present_letters = find_present_letters(api_url, seed, size)
    print(f"Present letters for seed {seed}: {present_letters}")

    # Step 2: Initialize current state and position dictionary
    current_state = ['_'] * size
    position_dict = {}

    # Step 3: Looping through each letter to assign appropriate index
    while True:
        # Try guessing each present letter
        for letter in present_letters:
            if guess_with_letter(api_url, letter, seed, size, current_state, position_dict):
                print(f"Correct word found: {''.join(current_state)}")
                break

        # Fill in unconfirmed positions with present letters
        fill_unconfirmed_positions(api_url, present_letters, size, current_state, position_dict, seed)

        # Stop if all positions are filled
        if len(position_dict) == size:
            break

    print(f"Final word: {''.join(current_state)}")

In [47]:

# Example usage
api_url = 'https://wordle.votee.dev:8000/random'
seed = 2321  # Example seed
word_size = 7  # Change this to any desired word size

main(api_url, seed, word_size)

Present letters for seed 2321: {'i', 'u', 'e', 's', 'r', 'o'}
Guess: iiiiiii, Response: [{'slot': 0, 'guess': 'i', 'result': 'present'}, {'slot': 1, 'guess': 'i', 'result': 'present'}, {'slot': 2, 'guess': 'i', 'result': 'present'}, {'slot': 3, 'guess': 'i', 'result': 'correct'}, {'slot': 4, 'guess': 'i', 'result': 'present'}, {'slot': 5, 'guess': 'i', 'result': 'present'}, {'slot': 6, 'guess': 'i', 'result': 'present'}]
Guess: uuuuuuu, Response: [{'slot': 0, 'guess': 'u', 'result': 'present'}, {'slot': 1, 'guess': 'u', 'result': 'present'}, {'slot': 2, 'guess': 'u', 'result': 'present'}, {'slot': 3, 'guess': 'u', 'result': 'present'}, {'slot': 4, 'guess': 'u', 'result': 'present'}, {'slot': 5, 'guess': 'u', 'result': 'correct'}, {'slot': 6, 'guess': 'u', 'result': 'present'}]
Guess: eeeeeee, Response: [{'slot': 0, 'guess': 'e', 'result': 'present'}, {'slot': 1, 'guess': 'e', 'result': 'correct'}, {'slot': 2, 'guess': 'e', 'result': 'present'}, {'slot': 3, 'guess': 'e', 'result': 'pres

#### Improvement on Steps:

1. Speed-Up: Find Present Letter can be simplified by looping from abcdef, ghijklm, ... , instead of using aaaaaa, bbbbbb every time

2. Adding validation to the final results


In [48]:
import requests
import string


def find_present_letters(api_url, seed, size):
    present_letters = set()
    alphabet = string.ascii_lowercase  # Initialize letters from a to z

    # Split the alphabet into groups for guessing
    for i in range(0, len(alphabet), size):
        # Determine the end index for slicing
        end_index = min(i + size, len(alphabet))
        group = alphabet[i:end_index]  # Take a group of letters
        
        # Fill the rest with 'a' if the group is shorter than size
        guess = ''.join(group) + 'a' * (size - len(group))
        result = guess_word(api_url, guess, seed, size)

        if result:
            for res in result:
                # Add only the letters that are marked as present or correct
                if res['result'] in ['present', 'correct']:
                    present_letters.add(res['guess'])

    return present_letters


In [49]:
def main(api_url, seed, size=5):
    # Step 1: Find present letters
    present_letters = find_present_letters(api_url, seed, size)
    print(f"Present letters for seed {seed}: {present_letters}")

    # Step 2: Initialize current state and position dictionary
    current_state = ['_'] * size
    position_dict = {}
    max_iterations = 100  # Prevent infinite loops
    iteration = 0

    # Step 3: Looping through each letter to assign appropriate index
    while True:
        iteration += 1
        if iteration > max_iterations:
            print("Reached maximum iterations. Exiting loop.")
            break
        
        # Try guessing each present letter
        for letter in present_letters:
            if guess_with_letter(api_url, letter, seed, size, current_state, position_dict):
                print(f"Correct word found: {''.join(current_state)}")
                break

        # Fill in unconfirmed positions with present letters
        fill_unconfirmed_positions(api_url, present_letters, size, current_state, position_dict, seed)

        # Stop if all positions are filled
        if len(position_dict) == size:
            break
        
    final_word = ''.join(current_state)
    print(f"Final word: {final_word}")

    # Step 4: Validate the final word with the API
    validation_result = guess_word(api_url, final_word, seed, size)
    print(validation_result)
    if validation_result:
        if all(res['result'] == 'correct' for res in validation_result):
            print("API validation is correct! The guessed word is confirmed.")
        else:
            print("API validation failed. The guessed word is incorrect.")
    else:
        print("Error in API validation.")

In [50]:

# Example usage
api_url = 'https://wordle.votee.dev:8000/random'
seed = 2321  # Example seed
word_size = 7  # Change this to any desired word size

main(api_url, seed, word_size)

Present letters for seed 2321: {'i', 'u', 'e', 's', 'r', 'o'}
Guess: iiiiiii, Response: [{'slot': 0, 'guess': 'i', 'result': 'present'}, {'slot': 1, 'guess': 'i', 'result': 'present'}, {'slot': 2, 'guess': 'i', 'result': 'present'}, {'slot': 3, 'guess': 'i', 'result': 'correct'}, {'slot': 4, 'guess': 'i', 'result': 'present'}, {'slot': 5, 'guess': 'i', 'result': 'present'}, {'slot': 6, 'guess': 'i', 'result': 'present'}]
Guess: uuuuuuu, Response: [{'slot': 0, 'guess': 'u', 'result': 'present'}, {'slot': 1, 'guess': 'u', 'result': 'present'}, {'slot': 2, 'guess': 'u', 'result': 'present'}, {'slot': 3, 'guess': 'u', 'result': 'present'}, {'slot': 4, 'guess': 'u', 'result': 'present'}, {'slot': 5, 'guess': 'u', 'result': 'correct'}, {'slot': 6, 'guess': 'u', 'result': 'present'}]
Guess: eeeeeee, Response: [{'slot': 0, 'guess': 'e', 'result': 'present'}, {'slot': 1, 'guess': 'e', 'result': 'correct'}, {'slot': 2, 'guess': 'e', 'result': 'present'}, {'slot': 3, 'guess': 'e', 'result': 'pres

#### Result: 8s V.S. 2.5s (faster)